In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np
from tqdm import tqdm

import math

In [ ]:
transform = transforms.Compose([
                    transforms.ToTensor(), 
                    # transforms.Normalize((0.5,), (0.5,)),
            ])

In [ ]:
def CreateDataLoaders(Option, p_BatchSize):
    if Option == 1:
        train_dataset = datasets.MNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.MNIST("./", train=False, transform = transform, download=True)

    elif Option == 2:
        train_dataset = datasets.FashionMNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.FashionMNIST("./", train=False, transform = transform, download=True)
    
    elif Option == 3:
        train_dataset = datasets.STL10("./", split="train", transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.STL10("./", split="test", transform = transform, download=True)
    
    elif Option == 4:
        train_dataset = datasets.SVHN("./", split="train", transform = transform, download=True)
        print(len(train_dataset))
        # print(int(len(train_dataset) * 0.85))
        # print(int(len(train_dataset) * 0.15))
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), math.ceil(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.SVHN("./", split="test", transform = transform, download=True)
    
    Train_DataLoader = torch.utils.data.DataLoader(train_subset, batch_size = p_BatchSize, shuffle = True)
    Val_DataLoader = torch.utils.data.DataLoader(val_subset, batch_size = p_BatchSize, shuffle = True)
    Test_DataLoader = torch.utils.data.DataLoader(test_dataset, batch_size = p_BatchSize, shuffle = True)

    return Train_DataLoader, Val_DataLoader, Test_DataLoader

In [ ]:
""

train_NumOfSamples = len(train_subset)
val_NumOfSamples = len(val_subset)
test_NumOfSamples = len(test_dataset)



In [ ]:
print(len(train_subset))
print(len(val_subset))
print(len(test_dataset))

51000
9000
10000


In [ ]:
dataiter = iter(Test_DataLoader)
images, labels = next(dataiter)

print(images.shape)
print(labels.shape)
print(labels)

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor([8, 2, 7, 4, 2, 2, 8, 4, 6, 1, 9, 2, 4, 3, 0, 6, 3, 9, 6, 9, 6, 1, 4, 3,
        1, 8, 7, 5, 3, 2, 3, 6, 1, 6, 6, 5, 8, 4, 2, 8, 2, 4, 8, 9, 1, 4, 3, 9,
        3, 7, 2, 0, 1, 5, 4, 6, 3, 7, 0, 8, 3, 4, 8, 3])


In [ ]:
class MLP(nn.Module):
    def __init__(self, InputDim, OutputDim):
        super(MLP, self).__init__()
        self.InputDim = InputDim
        self.OutputDim = OutputDim
        self.Linear1 = nn.Linear(InputDim, 800)
        self.Linear2 = nn.Linear(800, 800)
        self.Linear3 = nn.Linear(800, 800)
        self.Linear4 = nn.Linear(800, self.OutputDim)
        self.ActFunc = nn.functional.relu
        self.batch = nn.BatchNorm1d(800)
        # self.SftMax = nn.Softmax

    def forward(self, x):
        x = self.ActFunc(self.batch(self.Linear1(x)))
        x = self.ActFunc(self.batch(self.Linear2(x)))
        x = self.ActFunc(self.batch(self.Linear3(x)))

        # output = self.SftMax(self.Linear4(x))
        
        return self.Linear4(x)
    
    def InitWeights(self):
        torch.nn.init.uniform_(self.Linear1.weight,-0.5, 0.5)
        torch.nn.init.uniform_(self.Linear2.weight,-0.5, 0.5)

In [ ]:
def TrainModel(p_model, loss_criteria, Optimizer, device,  p_TrainDL):
    TrainCorr = 0
    TotNumOfSamples = 0
    ValAccuracy = float('inf')

    for images, labels in tqdm(p_TrainDL):
        Optimizer.zero_grad()

        images = images.reshape(images.shape[0], -1).to(device)
        labels = labels.to(device)

        pred = model(images)
        # print(type(pred))
        # print(pred.shape)
        # print(pred)
        # pred=pred.to(torch.float)
        # labels=labels.to(torch.float)
        predClass = torch.max(pred.data, 1)[1]
        # predClass = np.argmax(pred, axis=-1)
        # predClass = pred.max(1)
        # print("Pred", predClass)
        # print(type(labels))
        # print("labels", labels)

        # predClass=predClass.to(torch.float)
        loss = loss_criteria(pred, labels)
        TrainCorr += (predClass==labels).sum()
        TotNumOfSamples += len(labels)

        loss.backward()
        Optimizer.step()
    
    return TrainCorr/TotNumOfSamples

def EvaluateModel(p_model, p_loader, device):
    Correct = 0
    TotalNumOfSamples = 0

    for images, labels in tqdm(p_loader):
        images = images.reshape(images.shape[0], -1).to(device)
        labels = labels.to(device)

        pred = p_model(images)
        predClass = torch.max(pred.data, 1)[1]
        Correct += (predClass==labels).sum()
        TotalNumOfSamples += len(labels)
        # loss = loss_criteria(predClass, labels)
        # Accuracy += loss.cpu().data.item()

    return Correct/TotalNumOfSamples

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 100

In [ ]:
# -------------------------------------------- MNIST Dataset -----------------------------------------------------------
Option = 1
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_MNIST.pt"

model = MLP(images.shape[1]*images.shape[2]*images.shape[2], 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0
saved = False
if saved == True:
    model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)

    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

print()
Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
Test_Accuracy = EvaluateModel(model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy)

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

In [ ]:
print(model)

MLP(
  (Linear1): Linear(in_features=784, out_features=800, bias=True)
  (Linear2): Linear(in_features=800, out_features=800, bias=True)
  (Linear3): Linear(in_features=800, out_features=800, bias=True)
  (Linear4): Linear(in_features=800, out_features=10, bias=True)
  (batch): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


100%|██████████| 141/141 [00:01<00:00, 106.78it/s]


EPOCH -  0 . Train Accuracy =  tensor(0.7981, device='cuda:0') , Validation Accuracy =  tensor(0.9629, device='cuda:0')


100%|██████████| 141/141 [00:01<00:00, 109.91it/s]


EPOCH -  1 . Train Accuracy =  tensor(0.8321, device='cuda:0') , Validation Accuracy =  tensor(0.9697, device='cuda:0')


100%|██████████| 141/141 [00:01<00:00, 104.62it/s]


...

EPOCH -  7 . Train Accuracy =  tensor(0.8464, device='cuda:0') , Validation Accuracy =  tensor(0.9728, device='cuda:0')


100%|██████████| 141/141 [00:01<00:00, 106.64it/s]


EPOCH -  8 . Train Accuracy =  tensor(0.8464, device='cuda:0') , Validation Accuracy =  tensor(0.9786, device='cuda:0')


100%|██████████| 141/141 [00:01<00:00, 97.87it/s] 

EPOCH -  9 . Train Accuracy =  tensor(0.8476, device='cuda:0') , Validation Accuracy =  tensor(0.9752, device='cuda:0')


100%|██████████| 157/157 [00:01<00:00, 105.70it/s]

Test Accuracy =  tensor(0.7978, device='cuda:0')


In [ ]:
# -------------------------------------------- FMNIST Dataset -----------------------------------------------------------
Option = 2
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_FMNIST.pt"

model = MLP(784, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

print()
Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

100%|██████████| 141/141 [00:01<00:00, 114.44it/s]


EPOCH -  1 . Train Accuracy =  0.8465686440467834 , Validation Accuracy =  0.8668888807296753
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 109.60it/s]


EPOCH -  2 . Train Accuracy =  0.8898627758026123 , Validation Accuracy =  0.8792222142219543
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 111.22it/s]


...

EPOCH -  100 . Train Accuracy =  0.9976862668991089 , Validation Accuracy =  0.9024444818496704
Model Re-Saved



100%|██████████| 797/797 [00:07<00:00, 100.35it/s]


Train Accuracy =  0.9984509944915771


100%|██████████| 157/157 [00:01<00:00, 110.44it/s]

Test Accuracy =  0.8967999815940857
Generalization Gap =  0.10165101289749146


In [ ]:
# -------------------------------------------- STL 10 Dataset -----------------------------------------------------------
Option = 3
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_STL10.pt"

model = MLP(27648, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

print()
Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 12/12 [00:00<00:00, 37.45it/s]


EPOCH -  1 . Train Accuracy =  0.33858823776245117 , Validation Accuracy =  0.3933333158493042
Model Re-Saved


100%|██████████| 12/12 [00:00<00:00, 36.08it/s]


EPOCH -  2 . Train Accuracy =  0.4941176474094391 , Validation Accuracy =  0.41600000858306885
Model Re-Saved


...

EPOCH -  100 . Train Accuracy =  0.9981176853179932 , Validation Accuracy =  0.41600000858306885



100%|██████████| 67/67 [00:01<00:00, 36.50it/s]


Train Accuracy =  1.0


100%|██████████| 125/125 [00:03<00:00, 36.12it/s]

Test Accuracy =  0.41850000619888306
Generalization Gap =  0.5814999938011169


In [ ]:
# -------------------------------------------- SVHN Dataset -----------------------------------------------------------
Option = 4
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_SVHN.pt"

model = MLP(3072, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

print()
Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

Using downloaded and verified file: ./train_32x32.mat
73257


100%|██████████| 172/172 [00:02<00:00, 85.23it/s]


EPOCH -  1 . Train Accuracy =  0.6348044276237488 , Validation Accuracy =  0.7269997000694275
Model Re-Saved


100%|██████████| 172/172 [00:02<00:00, 81.55it/s]


EPOCH -  2 . Train Accuracy =  0.7535170912742615 , Validation Accuracy =  0.763035774230957
Model Re-Saved


...

EPOCH -  100 . Train Accuracy =  0.9893043041229248 , Validation Accuracy =  0.848757803440094



100%|██████████| 973/973 [00:12<00:00, 80.60it/s]


Train Accuracy =  0.9911832809448242


100%|██████████| 407/407 [00:04<00:00, 82.27it/s]

Test Accuracy =  0.8196834325790405
Generalization Gap =  0.1714998483657837
